In [1]:
import SimpleITK as sitk 
import numpy as np  
import cc3d
import pandas as pd
import numpy as np
import os

In [2]:
PSMA_SEGMENTATION_FOLDER = '/data/blobfuse/PSMA_PCA_LESIONS_SEGMENTATION/data_resampled_results/'
WORKING_FOLDER = "/home/jhubadmin/Desktop/segmentation_research/lymphoma-segmentation-dnn/"
RESULTS_FOLDER = "/data/blobfuse/PSMA_PCA_LESIONS_SEGMENTATION/data_resampled_results/check_val_results"

In [3]:
fold = 0
trainvalid_fpath = os.path.join(WORKING_FOLDER, 'data_split/train_filepaths.csv')
trainvalid_df = pd.read_csv(trainvalid_fpath)
train_df = trainvalid_df[trainvalid_df['FoldID'] != fold]
valid_df = trainvalid_df[trainvalid_df['FoldID'] == fold]

In [4]:
ct_files = valid_df["CTPATH"].values
pt_files = valid_df["PTPATH"].values
gt_files = valid_df["GTPATH"].values

In [5]:
pred_dir = "/data/blobfuse/PSMA_PCA_LESIONS_SEGMENTATION/data_resampled_results/check_val_results/predictions/fold0/unet/unet_fold0_randcrop128/" 
_files = sorted(os.listdir(pred_dir))

pred_files=[]
for i in range(len(_files)):
    pred_file = os.path.join(pred_dir,_files[i])
    pred_files.append(pred_file)

In [6]:
def get_spacing_from_niftipath(path):
    image = sitk.ReadImage(path)
    return image.GetSpacing()

In [7]:
def get_3darray_from_niftipath(
    path: str,
) -> np.ndarray:
    """Get a numpy array of a Nifti image using the filepath

    Args:
        path (str): path of the Nifti file

    Returns:
        np.ndarray: 3D numpy array for the image
    """
    image = sitk.ReadImage(path)
    array = np.transpose(sitk.GetArrayFromImage(image), (2,1,0))
    return array



In [8]:
ptarray = get_3darray_from_niftipath(pt_files[1])
predarray = get_3darray_from_niftipath(pred_files[1])
maskarray = get_3darray_from_niftipath(gt_files[1])

In [15]:
l_out,n = cc3d.connected_components(maskarray, connectivity=18, return_N=True) 

In [19]:
np.unique(l_out)

array([0, 1, 2, 3, 4, 5], dtype=uint16)

In [16]:
n

5

In [28]:
def calculate_lesionwise_suvmean_suvmax(
    ptarray: np.ndarray, 
    maskarray: np.ndarray,
    marker: str = 'SUVmean'
) -> np.float64:

    lesion_suv_means = []
    lesion_suv_max = []

    labels_out, num_lesions = cc3d.connected_components(maskarray, connectivity=18, return_N=True)
    
    for i in range(1, num_lesions+1):
        mask = np.zeros_like(labels_out)
        mask[labels_out == i] = 1
        prod = np.multiply(mask, ptarray)
        num_nonzero_voxels = len(np.nonzero(mask)[0])
        if marker == "SUVmean":
            lesion_suv_means.append(np.sum(prod)/num_nonzero_voxels)
        elif marker == "SUVmax":
            lesion_suv_max.append(np.max(prod))            

    if marker == "SUVmean":
        return lesion_suv_means
    elif marker =="SUVmax":
        return lesion_suv_max


In [29]:
def get_num_lesions (maskarray):
    _, num_lesions = cc3d.connected_components(maskarray, connectivity=18, return_N=True)
    return num_lesions

In [30]:
def calculate_lesionwise_tmtv(
    maskarray: np.ndarray,
    spacing: tuple
) -> np.float64:
    """Function to return the total metabolic tumor volume (TMTV) in cm^3 using 
    3D mask containing 0s for background and 1s for lesions/tumors
    Args:
        maskarray (np.ndarray): numpy ndarray for 3D mask image

    Returns:
        np.float64: 
    """
    voxel_volume_cc = np.prod(spacing) / 1000
    labels_out, num_lesions = cc3d.connected_components(maskarray, connectivity=18, return_N=True)
    
    if num_lesions == 0:
        return 0.0
    else:
        _, lesion_num_voxels = np.unique(labels_out, return_counts=True)
        lesion_num_voxels = lesion_num_voxels[1:]
        lesion_mtvs = voxel_volume_cc*lesion_num_voxels
    
    return lesion_mtvs


In [22]:
spacing = get_spacing_from_niftipath(gt_files[1])

In [31]:

def calculate_lesionwise_tlg(
    ptarray: np.ndarray,
    maskarray: np.ndarray,
    spacing: tuple
) -> np.float64:
    """Function to return the total lesion glycolysis (TLG) using a 3D PET image 
    and the corresponding 3D segmentation mask (containing 0s for background and
    1s for lesion/tumor)
    TLG = SUV1*V1 + SUV2*V2 + ... + SUVn*Vn, where SUV1...SUVn are the SUVmean 
    values of lesions 1...n with volumes V1...Vn, respectively

    Args:
        ptarray (np.ndarray): numpy ndarray for 3D PET image
        maskarray (np.ndarray): numpy ndarray for 3D mask image

    Returns:
        np.float64: total lesion glycolysis in cm^3 (assuming SUV is unitless)
    """
    voxel_volume_cc = np.prod(spacing)/1000 # voxel volume in cm^3

    labels_out, num_lesions = cc3d.connected_components(maskarray, connectivity=18, return_N=True)
    tlg = []
    if num_lesions == 0:
        return 0.0
    else:
        _, lesion_num_voxels = np.unique(labels_out, return_counts=True)
        lesion_num_voxels = lesion_num_voxels[1:]
        lesion_mtvs = voxel_volume_cc*lesion_num_voxels
        
        lesion_suvmeans = []
        for i in range(1, num_lesions+1):
            mask = np.zeros_like(labels_out)
            mask[labels_out == i] = 1
            prod = np.multiply(mask, ptarray)
            num_nonzero_voxels = len(np.nonzero(mask)[0])
            lesion_suvmeans.append(np.sum(prod)/num_nonzero_voxels)
        
        tlg = np.multiply(lesion_mtvs, lesion_suvmeans)
    return tlg

In [32]:
def calculate_lesion_metrics(lesion, ptarray, spacing, calculate_tmtv, calculate_suvmean_suvmax, calculate_tlg):
    """Helper function to calculate all metrics for a single lesion."""
    metrics = {
        'mtv': calculate_tmtv(lesion, spacing),
        'suvmean': calculate_suvmean_suvmax(ptarray, lesion, marker='SUVmean'),
        'suvmax': calculate_suvmean_suvmax(ptarray, lesion, marker='SUVmax'),
        'tlg': calculate_tlg(ptarray, lesion, spacing)
    }
    return metrics

def match_lesions(ptarray, ground_truth_mask, predicted_mask, spacing):
    gt_labeled, gt_num = cc3d.connected_components(ground_truth_mask, connectivity=18, return_N=True)
    pred_labeled, pred_num = cc3d.connected_components(predicted_mask, connectivity=18, return_N=True)

    gt_metrics = {}
    pred_metrics = {}
    matches = []

    for i in range(1, gt_num + 1):
        gt_lesion = gt_labeled == i
        gt_metrics[i] = calculate_lesion_metrics(gt_lesion, ptarray, spacing, calculate_lesionwise_tmtv, calculate_lesionwise_suvmean_suvmax, calculate_lesionwise_tlg)

    for j in range(1, pred_num + 1):
        pred_lesion = pred_labeled == j
        pred_metrics[j] = calculate_lesion_metrics(pred_lesion, ptarray, spacing, calculate_lesionwise_tmtv, calculate_lesionwise_suvmean_suvmax, calculate_lesionwise_tlg)

    for i in range(1, gt_num + 1):
        for j in range(1, pred_num + 1):
            #Check if there is an overlap between voxels then indicate that as a match
            if np.any((gt_labeled == i) & (pred_labeled == j)): 
                matches.append((i, j))

    return gt_metrics, pred_metrics, matches

gt_metrics, pred_metrics, matches = match_lesions(ptarray, maskarray, predarray, spacing)


In [23]:
gt_metrics

{1: {'mtv': array([2.34711902]),
  'suvmean': [1.797012045751014],
  'suvmax': [4.362869276409522],
  'tlg': array([4.21780116])},
 2: {'mtv': array([2.56444486]),
  'suvmean': [6.125233440988154],
  'suvmax': [15.206745068646311],
  'tlg': array([15.70782342])},
 3: {'mtv': array([2.73830553]),
  'suvmean': [1.0729957853875407],
  'suvmax': [2.35724704606916],
  'tlg': array([2.93819029])},
 4: {'mtv': array([6.12858856]),
  'suvmean': [4.792319369032199],
  'suvmax': [14.787530718404419],
  'tlg': array([29.37015368])},
 5: {'mtv': array([1.34742018]),
  'suvmean': [3.6453494240391504],
  'suvmax': [7.8069708453329465],
  'tlg': array([4.91181738])}}

In [24]:
pred_metrics

{1: {'mtv': array([0.65197751]),
  'suvmean': [3.8763949232313366],
  'suvmax': [6.364083131663564],
  'tlg': array([2.5273223])},
 2: {'mtv': array([3.12949203]),
  'suvmean': [5.754393860215095],
  'suvmax': [15.206745068646311],
  'tlg': array([18.00832974])},
 3: {'mtv': array([0.04346517]),
  'suvmean': [7.044459343326167],
  'suvmax': [7.044459343326167],
  'tlg': array([0.3061886])},
 4: {'mtv': array([2.04286285]),
  'suvmean': [3.001329394592281],
  'suvmax': [7.8069708453329465],
  'tlg': array([6.13130434])}}

In [37]:
g_out,n = cc3d.connected_components(maskarray, connectivity=18, return_N=True) 

In [38]:
p_out,n = cc3d.connected_components(predarray, connectivity=18, return_N=True)

In [47]:
b = np.any((p_out==2) & (g_out==2))

In [14]:
def lesionwise_dice(maskarray, predarray, g, p):
    g_out,_ = cc3d.connected_components(maskarray, connectivity=18, return_N=True) 
    p_out,_ = cc3d.connected_components(predarray, connectivity=18, return_N=True)
    
    g_mask = np.zeros_like(g_out)
    g_mask[g_out == g] = 1

    p_mask = np.zeros_like(p_out)
    p_mask[p_out == p] = 1

    # Calculate intersection
    intersection = p_mask[g_mask==1]

    # Compute Dice score
    dice = 2 * np.sum(intersection) / (np.sum(g_mask) + np.sum(p_mask))
    
    return dice

In [18]:
def categorize_by_size(mtv):
    if mtv < 1.5:
        return 'small'
    elif mtv < 4:
        return 'medium'
    else:
        return 'large'
    
def calculate_pres_recall(maskarray, predarray, g, p, metric_type):
    g_out,_ = cc3d.connected_components(maskarray, connectivity=18, return_N=True) 
    p_out,_ = cc3d.connected_components(predarray, connectivity=18, return_N=True)
    
    g_mask = np.zeros_like(g_out)
    g_mask[g_out == g] = 1

    p_mask = np.zeros_like(p_out)
    p_mask[p_out == p] = 1

    # Calculate intersection
    intersection = p_mask[g_mask==1]

    # Compute Dice score
    precision = np.sum(intersection) / np.sum(p_mask)
    recall = np.sum(intersection) / np.sum(g_mask)
    
    if metric_type =="Precision":
        return precision
    else:
        return recall

In [37]:
def write_p_r_to_df(filenames, predfiles, pt_files, gt_files):    

    columns = [
            'patient_filename', 'Precision', 'Recall'
        ]
    df = pd.DataFrame(columns=columns)
    
    for index in range(len(predfiles)):
        patient_filename = filenames[index]
        pt_array = get_3darray_from_niftipath(pt_files[index])
        mask_array = get_3darray_from_niftipath(gt_files[index])
        pred_array = get_3darray_from_niftipath(predfiles[index])
        
        spacing = get_spacing_from_niftipath(gt_files[index])

        gt_metrics, _, matches = match_lesions(pt_array, mask_array, pred_array, spacing)
        
        for gt_id in gt_metrics:
            row = {
                'patient_filename': patient_filename,
                }

            # Set ground truth metrics
            match = next((m for m in matches if m[0] == gt_id), None)
            if match:
                pred_id = match[1]
                row.update({'Precision': calculate_pres_recall(mask_array,pred_array, gt_id, pred_id, "Precision")})
                row.update({'Recall': calculate_pres_recall(mask_array,pred_array, gt_id, pred_id, "Recall")})
            else:
                row.update({'Precision': np.nan})
                row.update({'Recall': np.nan})
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    
    df.to_csv('pr_metrics.csv', index=False)
    return df

In [38]:
dfpr = write_p_r_to_df(_files, pred_files, pt_files, gt_files)

In [36]:
dfpr

,patient_filename,Precision,Recall
0,PSMA-01-003.nii.gz,0.878261,0.789062
1,PSMA-01-006.nii.gz,NaN,NaN
2,PSMA-01-006.nii.gz,0.777778,0.949153
3,PSMA-01-006.nii.gz,NaN,NaN
4,PSMA-01-006.nii.gz,1.000000,0.007092
5,PSMA-01-006.nii.gz,0.659574,1.000000
6,PSMA-01-008.nii.gz,0.857143,0.873786
7,PSMA-01-011.nii.gz,0.476190,1.000000
8,PSMA-01-014.nii.gz,0.560345,1.000000


In [69]:
def calculate_patient_level_dissemination(
    maskarray: np.ndarray,
    spacing: tuple
) -> np.float64:
    """Function to return the tumor dissemination (Dmax) using 3D segmentation mask
    Dmax = max possible distance between any two foreground voxels in a patient;
    these two voxels can come form the same lesions (in case of one lesion) 
    or from different lesions (in case of multiple lesions) 
   
    Args:
        maskarray (np.ndarray): numpy array for 3D mask image

    Returns:
        np.float64: dissemination value in cm
    """
    maskarray = maskarray.astype(np.int8)
    nonzero_voxels = np.argwhere(maskarray == 1)
    distances = np.sqrt(np.sum(((nonzero_voxels[:, None] - nonzero_voxels) * spacing)**2, axis=2))
    farthest_indices = np.unravel_index(np.argmax(distances), distances.shape)
    dmax = distances[farthest_indices]/10  # converting to cm
    del maskarray 
    del nonzero_voxels
    del distances 
    return dmax 

In [70]:
def calculate_patient_level_dice_score(
    gtarray: np.ndarray,
    predarray: np.ndarray, 
) -> np.float64:
    """Function to return the Dice similarity coefficient (Dice score) between
    2 segmentation masks (containing 0s for background and 1s for lesions/tumors)

    Args:
        maskarray_1 (np.ndarray): numpy ndarray for the first mask
        maskarray_2 (np.ndarray): numpy ndarray for the second mask

    Returns:
        np.float64: Dice score
    """
    dice_score = 2.0*np.sum(predarray[gtarray == 1])/(np.sum(gtarray) + np.sum(predarray))
    return dice_score

In [78]:
def create_and_save_dataframe(filenames, predfiles, pt_files, gt_files, spacing):
    columns = [
        'patient_filename', 'Number of gt_lesions', 'Number of pred lesions', 'Lesion ID', 'PLvl_Dice Score', 'Dmax'
    ]
    df = pd.DataFrame(columns=columns)
    
    for index in range(len(predfiles)):
        patient_filename = filenames[index]
        pt_array = get_3darray_from_niftipath(pt_files[index])
        mask_array = get_3darray_from_niftipath(gt_files[index])
        pred_array = get_3darray_from_niftipath(predfiles[index])

        gt_metrics, pred_metrics, matches = match_lesions(pt_array, mask_array, pred_array, spacing)
        dice_score = calculate_patient_level_dice_score(mask_array, pred_array)
        dmax = calculate_patient_level_dissemination(mask_array, spacing)
        num_lesions_gt = len(gt_metrics)
        num_lesions_pred = len(pred_metrics)

        for gt_id in gt_metrics:
            row = {
                'patient_filename': patient_filename,
                'Number of gt_lesions': num_lesions_gt,
                'Number of pred lesions': num_lesions_pred,
                'Lesion ID': gt_id,
                'Dmax':dmax,
                'PLvl_Dice Score': dice_score
                }

            # Set ground truth metrics
            row.update({f'{metric}_gt': gt_metrics[gt_id].get(metric, np.nan) for metric in ['suvmean', 'suvmax', 'mtv', 'tlg']})
            match = next((m for m in matches if m[0] == gt_id), None)
            if match:
                pred_id = match[1]
                row.update({f'{metric}_pred': pred_metrics[pred_id].get(metric, np.nan) for metric in ['suvmean', 'suvmax', 'mtv', 'tlg']})
                row.update({'lesionwise_dice': lesionwise_dice(mask_array,pred_array, gt_id, pred_id)})
            else:
                
                row.update({f'{metric}_pred': np.nan for metric in ['suvmean', 'suvmax', 'mtv', 'tlg']})
                row.update({'lesionwise_dice': np.nan})
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    
    df.to_csv('patient_lesion_metrics.csv', index=False)
    return df

In [79]:
df = create_and_save_dataframe(_files, pred_files, pt_files, gt_files, spacing)

In [77]:
df

,patient_filename,Number of gt_lesions,Number of pred lesions,Lesion ID,Dice Score,Dmax,suvmean_gt,suvmax_gt,mtv_gt,tlg_gt,suvmean_pred,suvmax_pred,mtv_pred,tlg_pred,lesionwise_dice
0,PSMA-01-003.nii.gz,1,2,1,0.779923,2.505833,[4.553546421752873],[13.247252461905719],[5.563541391664086],[25.333843996286],[4.92427421425143],[13.247252461905719],[4.998494219073202],[24.61395619306701],0.831276
1,PSMA-01-006.nii.gz,5,4,1,0.364389,18.152948,[1.797012045751014],[4.362869276409522],[2.3471190246082863],[4.217801160032461],NaN,NaN,NaN,NaN,NaN
2,PSMA-01-006.nii.gz,5,4,2,0.364389,18.152948,[6.125233440988154],[15.206745068646311],[2.5644448602201644],[15.707823415390743],[5.754393860215095],[15.206745068646311],[3.1294920328110485],[18.008329739199954],0.854962
3,PSMA-01-006.nii.gz,5,4,3,0.364389,18.152948,[1.0729957853875407],[2.35724704606916],[2.738305528709667],[2.938190291408874],NaN,NaN,NaN,NaN,NaN
4,PSMA-01-006.nii.gz,5,4,4,0.364389,18.152948,[4.792319369032199],[14.787530718404419],[6.12858856425497],[29.370153681308324],[7.044459343326167],[7.044459343326167],[0.04346516712237567],[0.30618860264445263],0.014085
5,PSMA-01-006.nii.gz,5,4,5,0.364389,18.152948,[3.6453494240391504],[7.8069708453329465],[1.347420180793646],[4.911817379994845],[3.001329394592281],[7.8069708453329465],[2.0428628547516565],[6.131304335086848],0.794872
6,PSMA-01-008.nii.gz,1,8,1,0.514286,2.424351,[7.760887960634833],[30.757347070604776],[4.476912213604694],[34.74481409938372],[7.590930280829957],[30.757347070604776],[4.563842547849445],[34.64381059341049],0.865385
7,PSMA-01-011.nii.gz,1,2,1,0.571429,0.878370,[2.8114684545499884],[4.258717648222057],[0.4346516712237567],[1.2220094623630249],[2.1952652444915906],[4.258717648222057],[0.9127685095698891],[2.0037689853251672],0.645161
8,PSMA-01-014.nii.gz,1,2,1,0.714286,1.990386,[4.485138644942813],[8.312285074704427],[2.8252358629544188],[12.67157455001522],[3.5606357927491534],[8.312285074704427],[5.041959386195578],[17.952581056075527],0.718232
